Vincent Earl Andrews 

All region test

In [13]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
import numpy as np
from astropy.io import fits
sns.set_context('talk')
plt.style.use('default')
from warnings import filterwarnings
filterwarnings('ignore')
from astropy.wcs import wcs
from time import sleep
import statistics

Functions used: 
> open_file : extracts data from fits file, removes Nans and removes 2nd frequency channel.
> kmeans : performs clustering 
> get_galcord : gets galactic coordinates using corresponding moment 0 map
> get_coordinates : i dont think i use this 
> get_centroid_radius : retruns apprximate radius of clump in a circular region about rhe centroid

In [88]:
def open_file(path):
    '''
    Function:
    path: Used for looping through several fits files with a common naming scheme as shown below
    
    This function opens fits files from the RAMPS data and extracts the data into a
    pandas dataframe
        - hdr.set was used to fill in headers that were empty and giving error messages 
        - the 2nd spectral channel is removed, should only be 1 channel for a map
        - nan values were dropped 
        - 
    '''
    print("opening file...")
    hdul = fits.open('D:/hf_vel/'+path+'_NH3_1-1_hf_width.fits')
    hdr = hdul[0].header
    hdr.set('BLANCK', 'none') # idk why blanck = -1 originally 
    hdr.set('NAXIS3',1) #i get an error message when this is gone idk
    d = hdul[0].data #3D with 2 channels ?
    data = np.swapaxes(d,0,2)
    hdul.close()
    coords = []
    values = []
    for (i, j, k), value in np.ndenumerate(data):
        coords.append((i, j, k))
        values.append(value)
    df = pd.DataFrame({'Coordinate': coords, 'Value': values})
    df[['X', 'Y', 'Z']] = pd.DataFrame(df['Coordinate'].tolist(), index=df.index)
    df = df.drop('Coordinate', axis=1)
    DF = df.dropna() #removing nan values 
    #reduce data frame to points of interest 
    lowHF = DF[(DF['Value'] <=100)]
    lowHF_XY = lowHF.drop(columns=["Z","Value"])
    return lowHF_XY, lowHF

def Kmeans(data, n, region):
    print("performing clustering...")
    km = KMeans(n_clusters=n,init='k-means++',n_init = 1000, tol = 1e-4)
    km.fit(data)
    labels = km.predict(data)
    centroids = km.cluster_centers_
    centroid_centers = km.cluster_centers_
    
    # This gets the index of each value per clump
    cluster_map = pd.DataFrame()
    cluster_map['data_index'] = data.index.values
    cluster_map['Cluster'] = labels
    new_map = cluster_map.set_index('data_index') # change random index to cluster index

    # getting coordinates at each index per clump
    # join dataframes where indicies are the same
    cords = pd.concat([lowHF,new_map], axis = 1, join = 'inner')
    cords.insert(0, 'Region', region)
    cords.drop(['Z'], axis = 1) # remove 3rd axis
    
    columns  = ['Region', 'Cluster', 'Value','X','Y']
    ordered_cords = cords[columns]
    #print(ordered_cords)
    #print(len(ordered_cords))
    # yeah it works 0_0 this gives the centorid number each point is assigned to
    return labels, centroids, centroid_centers, ordered_cords

def get_galcord(filepath):
    '''
    Definition:
    filepath: naming scheme for mom0 files follows the mom0_head variable below 
    mom0: The moment zero map is the total integrated intensity of the 
    spectral line being observed as some frequency, f.
    
    Function:
    Computes the galactic coordinates (GLON and GLAT) and the right ascension, 
    declination coordinates alongside the pixel coordinates for the center of 
    each clump found
    '''
    print("getting galactic cooridnates...")
    mom0_head = fits.getheader('D:/hf_vel/'+filepath+'_NH3_1-1_mom0.fits')
    #using world coordinates system conversion (for all points)
    w = wcs.WCS(mom0_head)
    sky = w.pixel_to_world(lowHF.X, lowHF.Y)
    equ = sky.transform_to('icrs')
    lowHF['GLON'] = sky.l.value
    lowHF['GLAT'] = sky.b.value
    lowHF['RA'] = equ.ra.value
    lowHF['DEC'] = equ.dec.value
    
    #using wcs for conversion of center points only
    df_cent = pd.DataFrame(centroids, columns=['x_center', 'y_center'])
    sky = w.pixel_to_world(df_cent.x_center, df_cent.y_center)
    equ_cent = sky.transform_to('icrs')
    #creating galactic coordinate dataframe 
    df_cent['GLON'] = sky.l.value
    df_cent['GLAT'] = sky.b.value
    df_cent['RA'] = equ_cent.ra.value
    df_cent['DEC'] = equ_cent.dec.value
    
    '''
    Returns Dataframe:
    columns = [x_center, y_center, GLON, GLAT, RA, DEC]
    length of number of clumps per region, L. Total length should be number of clumps
    across all regions
    '''
    return df_cent

def get_centroid_radius(X,Y,center_hk):
    '''
    This function takes (X, Y): the pixel position of every point within a clump
    and center_hk: the center coordinate of the clump
    
    The radius is then computed for every set of points from the 
    2D distance formula
    
    The average distance is then selected as the clump radius, This works fine 
    if the clump is spherical
    '''
    print("Calculating radius of clumps...")
    possible_radii = [] # stores all radius values calculated
    h = center_hk[0] # h is x center cord, k is y center cord
    k = center_hk[1]
    
    # compute distance for each point to the center
    for x_pos in X:
        for y_pos in Y:
            d = np.sqrt((x_pos-h)**2+(y_pos-k)**2)
            possible_radii.append(d)  
    # get 
    rr = max(possible_radii)
    radii.append(rr)
    #yeahhhhhh it worked (:
    return radii

def jeans_mass_T(c):
    # c is the cords df containing all cords in each centroid 
    '''
    Definition:
    Jeans critical mass: The mass of a clump of gas at which gravitational energy exceeds
    the outward turbulence/pressure. At this mass, the cloud of gas will likely collapse
    to form a star.
    
    Function:
    Calculates the critical Jeans mass of a clump
    The equation for this depends on gas temperature
        T: Gas temperature [K]
        m: mass of particle in gas [kg]
        n: gas number = density/mean mass per particle [?]
        k: boltzman constant
        G: Gravitational constant 
    '''
    return ((375 * k**3) / (4 * pi * m**4 * G**3)) * (T**3 / n)**(1/3)

def jeans_mass_hf():
    '''
    Calculates the critical Jeans mass of a clump
    The equation for this depends on the sound speed, cs
        cs: sound speed of ammonia 
        rho: density
        G: Gravitational constant 
    '''
    return (pi/6) * ((cs**3) / (G**(3/2) * rho**(1/2)))
    
def ammonia_mass():
    '''
    Definition: 
    The clump mass can be determined by looking at abundance ratios of different elements 
    in our target region. Molecular hydrogen is one of the most abundance; therefore,
    we can look at the abudnace of ammonia wrt to hydrogen to roughly estimate the mass.
    
    Function:
    Calculates the clump mass using abundance ratios between
    hydrogen and ammonia
        mu: molecular weight per hydrogen 
        mH: atomic hydrogen mass
        **Apix: area of a pixel in physical coordinates** need distance resolved 
        N_NH3: ammonia column density
        **X_NH3: ammonia abundance wrt hydrogen** need to find   
    '''
    mu = 2.8 # kauffmann et al 2009
    mH = 1.6735575e-27 # [kg]
    
    # get mass per pixel
    M = 0
    
    # multiply by how many pixel in each cluster
    # get how many regions have clump designation, N, 
    # then multiply pixel mass times num of pixels
    
    # get mass in solar masses 
    Msun = 1.91e30 # mass of sun [kg]
    clump_masses = [] # list of all clump masses
    clump_mass = M / Msun
    
def avg_hf():
    '''
    This calculates the average hyperfine width of ammonia per clump
    
    '''

In [92]:
#coordinates for center of each clump 
CENT_LON = []
CENT_LAT = []

all_clump_designations = pd.DataFrame() # this stores all clump cords and their centroid number
'''
paths = ['L47','L43','L41','L40_5','L40','L39_5','L39','L38_5','L38','L37_5',
          'L37','L36_5','L36','L35_5','L35','L34_5','L34','L33_5','L33','L32_5'
          ,'L32','L31_5','L31','L30_5','L30','L29_5','L29','L28_5','L28','L27_5'
          ,'L27','L26_5','L26','L25_5','L25','L24_5','L24','L23_5','L23','L22_5'
          ,'L22','L21_5','L21','L20_5','L20','L19_5','L19','L18_5','L18','L17_5','L17',
          'L16_5','L15_5','L15','L14_5','L14','L13_5','L13','L12_5','L12',
          'L11_5','L11','L10_5','L10']
'''
paths = ['L19_5','L19','L18_5','L18','L17_5','L17',
          'L16_5','L15_5','L15','L14_5','L14','L13_5','L13','L12_5','L12',
          'L11_5','L11','L10_5','L10']

n_regions = len(path)
for L in paths:
    # PART 1: Opening file and processing data
    '''
    final_n = [6,16,6,7,5,6,23,26,26,26,26,42,62,40,41,
                41,3,25,19,15,69,50,93,89,93,80,39,92,60,40,30
                ,98,35,39,88,80,93,92,99,9,6,17,19,11,16,84,74,
                49,2,5,33,41,8,60,60,40,35,35,50,30,18,40,40,20]
    '''
    final_n = [84,74,49,2,5,33,41,8,60,60,40,35,35,50,30,18,40,40,20]
    i = paths.index(L)
    # usually final_n is a list corresponding to each path
    n = final_n[i]
    lowHF_XY, lowHF = open_file(L) 
  
    # PART 2: Performing clustering
    labels, centroids, centroid_centers, clump_designations = Kmeans(lowHF_XY,n, L) # centroid_centers are in pixel cords here
    #clump_radius = get_centroid_radius()
 
    # PART 3: Extract coordinates of clumps
    df_centroids = get_galcord(L) # df_centroids has XY, RA/DEC, l/b
 
    # PART 4: Plotting
    cmap = 'viridis' #colormap 
    
    #axis 2 is the galactic coordinates 
    plt.scatter(lowHF.GLON, lowHF.GLAT,s = 500, c=labels, cmap=cmap, vmin = 0, vmax = max(labels) * 1.3)
    plt.gca().invert_xaxis()
    plt.scatter(df_centroids.GLON, df_centroids.GLAT, marker='*', c='gold', alpha=1, s=800,edgecolor = 'black', label = 'Clump Centers')
    # idk what this is but im scared of deleting stuff
    #for i in range(df_centroids.shape[0]): #this numbers points 
    #  plt.scatter(df_centroids.GLON[i], df_centroids.GLAT[i], marker='$%d$' % i, s=40, edgecolor ='darkorchid')
    f = 48
    font = {'fontname':'Calibri'}
    plt.xlabel('Galactic Longitude', fontsize = f, **font)
    plt.ylabel('Galactic Latitude', fontsize = f, **font)
    plt.legend(prop = { "size": f-12 })
    plt.title("Region "+L+": Gas Clump Distribution", fontsize = f, **font)
    plt.xticks(fontsize=f)
    plt.yticks(fontsize=f)
    #need to invert image so longitude reflects hyperfine width image
    figure = plt.gcf()
    figure.set_size_inches(24,16)
    print("Saving Figure", '\n')
    plt.savefig('D:/clustering_plots/'+L+'clustering.png')
    plt.clf()
    '''
       # PART 5: Exporting data
    latcord = df_centroids.RA.tolist()
    loncord = df_centroids.DEC.tolist()
    for la in latcord:
        CENT_LAT.append(la)
    for lo in loncord:
        CENT_LON.append(lo)
    '''
    # append to all clump dataframe  
    all_clump_designations = pd.concat([all_clump_designations, clump_designations], ignore_index=True)

all_clump_designations.to_csv("D:/clump_designations/cutoff.csv",index = False)

opening file...
performing clustering...
getting galactic cooridnates...
Saving Figure 

opening file...
performing clustering...
getting galactic cooridnates...
Saving Figure 

opening file...
performing clustering...
getting galactic cooridnates...
Saving Figure 

opening file...
performing clustering...
getting galactic cooridnates...
Saving Figure 

opening file...
performing clustering...
getting galactic cooridnates...
Saving Figure 

opening file...
performing clustering...
getting galactic cooridnates...
Saving Figure 

opening file...
performing clustering...
getting galactic cooridnates...
Saving Figure 

opening file...
performing clustering...
getting galactic cooridnates...
Saving Figure 

opening file...
performing clustering...
getting galactic cooridnates...
Saving Figure 

opening file...
performing clustering...
getting galactic cooridnates...
Saving Figure 

opening file...
performing clustering...
getting galactic cooridnates...
Saving Figure 

opening file...
perfo

<Figure size 2400x1600 with 0 Axes>

In [4]:
# PART 5: Exporting data to csv
print("extracting coordinates to csv")
clmns = ['LON','LAT']
CENT_ALL = pd.DataFrame(list(zip(CENT_LAT,CENT_LON)),columns = clmns)
# uncomment to save csv file
CENT_ALL.to_csv('C:/Users/vince/Desktop/AmmoniaHF/data/L13_cords_test.csv',index = False)

extracting coordinates to csv
           LON        LAT
0   273.888013 -17.648951
1   273.628218 -17.854243
2   273.487630 -17.489670
3   273.460620 -18.030340
4   273.041078 -17.677414
..         ...        ...
57  273.433127 -17.194529
58  273.694373 -17.545121
59  273.616267 -17.736206
60  273.558070 -17.919612
61  273.405371 -17.309312

[62 rows x 2 columns]


In [6]:
'''
CODING GRAVEYARD  <3

def get_coordinates(Data,data_labels,centroid_num,center):
    # this function should get the coordinates of all points for each clump
    # need to do this before getting average hf_wdith and Ntot
    print("getting clump center cooridnates...")
    l = [].clear()
    b = [].clear()
    coordinates = lowHF[labels == centroid_num]
    #l = coordinates['GLON'].values.tolist()
    #b = coordinates['GLAT'].values.tolist()
    #coordinate_dictionary["Xcord"+str(centroid_num)] = X_cord
    #coordinate_dictionary["Ycord"+str(centroid_num)] = Y_cord
    #get_centroid_radius(l,b,center)
    return coordinates


    def get_coordinates(data_labels,centroid_num):
    # this function should get the coordinates of all points for each clump
    # need to do this before getting average hf_wdith and Ntot
    print("getting clump center cooridnates...")
    l = [].clear()
    b = [].clear()
    coordinates = lowHF[data_labels == centroid_num] # idk if this works
    hf = coordinates['value'].values.tolist()
    x = coordinates['X'].values.tolist()
    y = coordinates['Y'].values.tolist()
    
    print(hf)
    print(statistics.mean(hf))
    #coordinate_dictionary["Xcord"+str(centroid_num)] = X_cord
    #coordinate_dictionary["Ycord"+str(centroid_num)] = Y_cord
    #get_centroid_radius(l,b,center)
    return coordinates

'''

'\nCODING GRAVEYARD  <3\n\ndef get_coordinates(Data,data_labels,centroid_num,center):\n    # this function should get the coordinates of all points for each clump\n    # need to do this before getting average hf_wdith and Ntot\n    print("getting clump center cooridnates...")\n    l = [].clear()\n    b = [].clear()\n    coordinates = lowHF[labels == centroid_num]\n    #l = coordinates[\'GLON\'].values.tolist()\n    #b = coordinates[\'GLAT\'].values.tolist()\n    #coordinate_dictionary["Xcord"+str(centroid_num)] = X_cord\n    #coordinate_dictionary["Ycord"+str(centroid_num)] = Y_cord\n    #get_centroid_radius(l,b,center)\n    return coordinates\n\n\n    def get_coordinates(data_labels,centroid_num):\n    # this function should get the coordinates of all points for each clump\n    # need to do this before getting average hf_wdith and Ntot\n    print("getting clump center cooridnates...")\n    l = [].clear()\n    b = [].clear()\n    coordinates = lowHF[data_labels == centroid_num] # idk i